In [ ]:
!pip3 install pandas
!pip3 install numpy
!pip3 install scikit-learn
!pip3 install matplotlib
!pip3 install seaborn

Importaciones

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Carga de los datos y filtrado

In [ ]:
wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip")

# Apartado 1
EDA o Análisis Exploratorio de los Datos

In [ ]:
# Elimina todas las variables meteorológicas que no correspondan a la localización de Sotavento (la localización 13)
wind_ava = wind_ava.filter(regex='^(datetime|energy|.*\.13)$')

# Número de instancias y características
print(f'Número de instancias: {wind_ava.shape[0]}')
print(f'Número de características: {wind_ava.shape[1]}')

# Variables categóricas y numéricas
categorical_vars = wind_ava.select_dtypes(include=['object']).columns
numerical_vars = wind_ava.select_dtypes(include=['int64', 'float64']).columns
print(f'Variables categóricas: {categorical_vars}')
print(f'Variables numéricas: {numerical_vars}')

# Comprueba si hay valores faltantes y qué variables los tienen
missing_values = wind_ava.isnull().sum()
print(f'Valores faltantes por variable:\n{missing_values[missing_values > 0]}')

# Columnas constantes
constant_columns = [col for col in wind_ava.columns if wind_ava[col].nunique() <= 1]
print(f'Columnas constantes: {constant_columns}')

# Elimina las columnas constantes
wind_ava = wind_ava.loc[:, wind_ava.apply(pd.Series.nunique) != 1]

# Determinar si es un problema de regresión o clasificación
if wind_ava['energy'].nunique() > 2:
    print('Es un problema de regresión')
else:
    print('Es un problema de clasificación')


### Matriz de correlación
Podemos realizar una matriz de correlación para ver que variables están fuertemente relacionadas

In [ ]:
# Realizamos un analisis de la correlación de las variables númericas
correlation = wind_ava[numerical_vars].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, fmt=".2f", cmap='coolwarm')
plt.show()

Con estos resultados podemos ver que tanto las variables `` lai_lv.13`` y `` lai_hv.13 ``, como las variables `` v10.13 `` y `` v10n.13``, tienen una correlacion de 1. Por lo tanto, podemos quitar una de cada pareja, ya que dan información redundante.

In [ ]:
# Elimina las variables 'lai_hv.13' y 'v10n.13'
if 'lai_hv.13' in wind_ava.columns and 'v10n.13' in wind_ava.columns:
    wind_ava = wind_ava.drop(['lai_hv.13', 'v10n.13'], axis=1)

# Vuelve a definir las variables numéricas
numerical_vars = wind_ava.select_dtypes(include=['int64', 'float64']).columns

# Realizamos un analisis de la correlación de las variables númericas
correlation = wind_ava[numerical_vars].corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation, annot=True, fmt=".2f", cmap='coolwarm')
plt.show()

Podemos observar que sigue habiendo variables muy correlacionadas entre ellas, llegando a haber resultados de 0.99, pero en ningún caso aparece un 1 fuera de la diagonal. Además, con esta matriz podemos ver que la variable que más correlación tiene con la energía producida es `` p59.162 `` (Vertical integral of divergence of kinetic energy) con un grado casi del 50%.


In [ ]:
import matplotlib.pyplot as plt

# Para cada variable numérica, crear un histograma
for col in numerical_vars:
    plt.figure(figsize=(10, 6))
    wind_ava[col].hist(bins=30, alpha=0.5, label=col)
    plt.title(f'Distribución de {col}')
    plt.xlabel(col)
    plt.ylabel('Frecuencia')
    plt.legend(loc='upper right')
    plt.show()

Estos histogramas nos ayudan a conocer mejor como se distribuyen los datos  y qué valores mínimos y máximos toman nuestras variables.

In [ ]:
# Asegurarse de que 'datetime' es de tipo datetime
wind_ava['datetime'] = pd.to_datetime(wind_ava['datetime'])

# Lista de colores para cada año
colors = ['blue', 'green', 'red', 'purple', 'orange']

# Iterar sobre cada año único en los datos
for year, color in zip(wind_ava['datetime'].dt.year.unique(), colors):
    # Filtrar los datos para el año actual
    filtered_data = wind_ava[wind_ava['datetime'].dt.year == year]
    
    # Agrupar por mes y calcular la media de 'energy' para el año actual
    monthly_energy = filtered_data.groupby(filtered_data['datetime'].dt.to_period('M')).mean()
    
    # Crear un gráfico para el año actual
    plt.figure(figsize=(12, 6))
    
    # Trazar la media mensual de 'energy' para el año actual con el color correspondiente
    plt.plot(monthly_energy.index.to_timestamp(), monthly_energy['energy'], color=color)
    
    # Configurar etiquetas y título
    plt.xlabel('Fecha')
    plt.ylabel('Energía Media')
    plt.title(f'Media Mensual de Energía para el Año {year}')
    
    # Mostrar el gráfico
    plt.show()

Como vemos la mayoría de los meses en el año que menos energía se producce son en julio y agosto, mientras que cuando más energía se produce suele ser noviembre o marzo.

In [ ]:
# Para cada variable numérica, visualizar los outliers
for col in numerical_vars:
    Q1 = wind_ava[col].quantile(0.25)
    Q3 = wind_ava[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = wind_ava[(wind_ava[col] < (Q1 - 1.5 * IQR)) | (wind_ava[col] > (Q3 + 1.5 * IQR))]
    print(f"Outliers en {col}: {outliers[col].count()}")

# Lista de variables para las que visualizar outliers
variables = ['energy', 'p55.162.13', 'cape.13', 'p59.162.13', 'sp.13', 'iews.13', 'inss.13', 'fsr.13']

# Para cada variable, creamos un gráfico de caja
for var in variables:
    plt.figure(figsize=(10, 4))
    boxplot = plt.boxplot(wind_ava[var], vert=False)
    plt.setp(boxplot['boxes'], color='blue')
    plt.title(f'Boxplot de {var}')
    plt.show()

Los outliers identifican el número de datos que están fuera de la distribución. Nos ayudan a identificar si se han producido errores de medición, anomalías o cambios en la distribución de los datos. Con estos resultados podemos ver que las variables con el mayor número de outliers son cape.13 y inss.13, con 697 y 431 outliers respectivamente.

# Apartado 2
La evaluación ``outer`` del modelo se llevará a cabo por medio de la evualuación train/test, ya que disponemos de un gran número de datos. Por otro lado, en la evaluavión ``inner`` se utilizará más de un método de evaluación con el fin de probarlos y comparar sus resultados. Esto incluye train/test, evaluación cruzada, y el uso de Grid-search (para ajustar múltiples hiperparámetros)

# Apartado 3
En primer lugar, y usando el método KNN, vamos a comprobar cuál es el mejor método de escalado para nuestros datos. Para ello nos vamos a basar en el error cuadrático medio negativo (o RMSE), por lo que el modelo con el resultado más pequeño será el modelo más preciso

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

# Quitamos "energy" de las variables numéricas, ya que querremos usarla como variable objetivo
numerical_vars = numerical_vars.drop('energy')

# Dividir los datos en conjuntos de entrenamiento y prueba (evaluación externa)
X_train, X_test, y_train, y_test = train_test_split(wind_ava[numerical_vars], wind_ava['energy'], test_size=1/3, random_state=100472166, shuffle=False)

# Definir los métodos de escalado para evaluar
scalers = {
    'StandardScaler': StandardScaler(),
    'MinMaxScaler': MinMaxScaler(),
    'RobustScaler': RobustScaler()
}

# Evauluación interna con 3-fold time series cross-validation
inner = TimeSeriesSplit(n_splits=3)

# Diccionario para almacenar los resultados de la evaluación interna
inner_scores = {}

# Pipeline para StandardScaler
knn_standard = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsRegressor())
])

scores_std = cross_val_score(knn_standard, X_train, y_train, cv=inner, scoring='neg_root_mean_squared_error')
inner_scores['StandardScaler'] = -scores_std.mean()

# Pipeline para MinMaxScaler
knn_minmax = Pipeline([
    ('scaler', MinMaxScaler()),
    ('knn', KNeighborsRegressor())
])
scores_minmax = cross_val_score(knn_minmax, X_train, y_train, cv=inner, scoring='neg_root_mean_squared_error')
inner_scores['MinMaxScaler'] = -scores_minmax.mean()

# Pipeline para RobustScaler
knn_robust = Pipeline([
    ('scaler', RobustScaler()),
    ('knn', KNeighborsRegressor())
])
scores_robust = cross_val_score(knn_robust, X_train, y_train, cv=inner, scoring='neg_root_mean_squared_error')
inner_scores['RobustScaker'] = -scores_robust.mean()

# Imprimimos los resultados
for scaler_name, score in inner_scores.items():
    print(f"Variable Name: {scaler_name}")
    print(f"Variable Score: {score}")
    print()

# Elegimos el mejor método de escalado según el RMSE
mejor_scaler = min(inner_scores, key=inner_scores.get)
print(f"El mejor método de escalado es {mejor_scaler} con un RMSE de {inner_scores[min(inner_scores, key=inner_scores.get)]}")

# Apartado 4
De manera similar, ahora evaluaremos y mediremos los tiempos de distintos métodos. Primero lo haremos con sus hiperparámetros por defecto, como se muestra a continuación

In [ ]:
import time
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR
from sklearn.dummy import DummyRegressor

# Crear instancias de los modelos
knn = KNeighborsRegressor()
tree = DecisionTreeRegressor()
linear = LinearRegression()
lasso = Lasso()
svm = SVR()
dummy = DummyRegressor(strategy='mean')

# Crear listas para almacenar los tiempos de entrenamiento y los errores
times = {name: [] for name in scalers}
errors = {name: [] for name in scalers}

models = [knn, tree, linear, lasso, svm, dummy]
model_names = ['KNN', 'Árboles de Regresión', 'Regresión Lineal', 'Regresión Lasso', 'SVM', 'Dummy']

# Entrenar y evaluar los modelos con diferentes métodos de escalado
for scaler_name, scaler in scalers.items():
    if scaler is not None:
        # Escalar los datos de entrenamiento
        X_train_scaled = scaler.fit_transform(X_train)
        # Escalar los datos de prueba
        X_test_scaled = scaler.transform(X_test)
    else:
        # Si no se utiliza escalado, usar los datos originales
        X_train_scaled = X_train
        X_test_scaled = X_test

    for model, name in zip(models, model_names):
        start_time = time.time()
        model.fit(X_train_scaled, y_train)
        end_time = time.time()
        train_time = end_time - start_time
        times[scaler_name].append(train_time)

        y_pred = model.predict(X_test_scaled)
        error = mean_squared_error(y_test, y_pred)
        errors[scaler_name].append(error)

        print(f"Modelo: {name}, Escalado: {scaler_name}")
        print(f"Tiempo de entrenamiento: {train_time} segundos")
        print(f"Error cuadrático medio: {error}")
        print("------------------------")

### GridSearch
Ahora volveremos a evaluar los modelos, pero esta vez probando distintos valores para los hiperparámetros y mediante el uso de un procedimiento sistemático como ``GridSearchCV``

In [ ]:
from sklearn.model_selection import GridSearchCV

# Hiperparámetros
knn_params = {
    'n_neighbors': list(range(1, 31)),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
}
tree_params = {
    'max_depth': [10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2],
    'max_leaf_nodes': [10, 20, 30],
    'min_impurity_decrease': [0.0, 0.1],
    'criterion': ['squared_error'],
    'splitter': ['best'],
    'min_weight_fraction_leaf': [0.0],
}

linear_params = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
}
lasso_params = {
    'alpha': [0.1, 0.5, 1.0, 5.0, 10.0, 50.0, 100.0],
    'fit_intercept': [True, False],
    'max_iter': [10000],
    'precompute': [True, False],
    'tol': [1e-4, 1e-3, 1e-2, 1e-1],
    'warm_start': [True, False],
    'positive': [True, False],
    'selection': ['cyclic', 'random'],
    'random_state': [100472166]
}
svm_params = {
    'C': [0.001, 0.1, 1.0, 10.0, 100.0, 1000.0],
    'shrinking': [True, False],
    'gamma':['scale', 'auto'],
    'degree': [2,3,4,5]
}

# Realizamos GridSearchCV para cada modelo
knn_grid = GridSearchCV(KNeighborsRegressor(), knn_params)
tree_grid = GridSearchCV(DecisionTreeRegressor(), tree_params)
linear_grid = GridSearchCV(LinearRegression(), linear_params)
lasso_grid = GridSearchCV(Lasso(), lasso_params)
svm_grid = GridSearchCV(SVR(), svm_params)

# Ajusta los modelos con los mejores hiperparámetros
knn_grid.fit(X_train_scaled, y_train)

# Obtenemos los mejores hiperparámetros y puntuaciones para KNN
best_knn_params = knn_grid.best_params_
best_knn_score = knn_grid.best_score_
y_pred = knn_grid.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
print("KNN:")
print("Mejores Hiperparámetros:", best_knn_params)
print("Mejor Puntuación:", best_knn_score)
print("Error cuadrático medio:", mse)
print("------------------------")

# Obtenemos los mejores hiperparámetros y puntuaciones para árboles de decisión
tree_grid.fit(X_train_scaled, y_train)

best_tree_params = tree_grid.best_params_
best_tree_score = tree_grid.best_score_
y_pred = tree_grid.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
print("Árbol de Decisión:")
print("Mejores Hiperparámetros:", best_tree_params)
print("Mejor Puntuación:", best_tree_score)
print("Error cuadrático medio:", mse)
print("------------------------")

# Obtenemos los mejores hiperparámetros y puntuaciones para regresión lineal
linear_grid.fit(X_train_scaled, y_train)

best_linear_params = linear_grid.best_params_
best_linear_score = linear_grid.best_score_
y_pred = linear_grid.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
print("Regresión Lineal:")
print("Mejores Hiperparámetros:", best_linear_params)
print("Mejor Puntuación:", best_linear_score)
print("Error cuadrático medio:", mse)
print("------------------------")

# Obtenemos los mejores hiperparámetros y puntuaciones para Lasso
lasso_grid.fit(X_train_scaled, y_train)

best_lasso_params = lasso_grid.best_params_
best_lasso_score = lasso_grid.best_score_
y_pred = lasso_grid.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
print("Lasso:")
print("Mejores Hiperparámetros:", best_lasso_params)
print("Mejor Puntuación:", best_lasso_score)
print("Error cuadrático medio:", mse)
print("------------------------")

# Obtenemos los mejores hiperparámetros y puntuaciones para SVM
svm_grid.fit(X_train_scaled, y_train)

best_svm_params = svm_grid.best_params_
best_svm_score = svm_grid.best_score_
y_pred = svm_grid.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
print("SVM:") 
print("Mejores Hiperparámetros:", best_svm_params)
print("Mejor Puntuación:", best_svm_score)
print("Error cuadrático medio:", mse)
print("------------------------")

Basándonos en los resultados obtenidos, llegamos a las siguientes conclusiones:

En términos de puntuación, SVM obtuvo la más alta con un 0.65, lo que indica que es el modelo que mejor se ajusta a los datos. Observamos además que el error cuadrático medio es mucho menor ahora que cuando se usaban los parámetros por defecto, llegando a bajar de 427590 a 126242. No sólo eso, si no que ahora el error cuadrático medio de SVM también es el menor de todo el ajuste de hiperarámetros. Por todas estas razones consideramos que es el mejor modelo para usar con nuestros datos.

En cuanto a velocidad, el método más rápido obviamente es el "Dummy" con 0.0 segundos (es decir, es un método prácticamente instantáneo para este conjunto de datos). Sin embargo, los resultados del resto de métodos son obviamente mejores que los obtenidos con un "DummyRegressor". Si bien es cierto que este tipo de regresor nos permite ahorrar tiempo, el error cuadrático medio es mucho más alto que el de los demás.

En general, existe un equilibrio entre el tiempo de ejecución y la mejora de resultados. Por ejemplo, los árboles de regresión tardan más que el método KNN, pero al mismo tiempo KNN muestra peores resultados que los árboles de regresióm. Esto en nuestro análisis de los datos no se cumple siempre de todos modos, ya que el método de regresión lineal es más rápido que varios de los demás y aun así presenta el mejor resultado.

El ajuste de hiperparámetros nos ha ayudado por lo tanto a mejorar el rendimiento y la precisión de los modelos en comparación con los valores por defecto. Hay modelos que han reducido poco su error (por ejemplo en regresión lineal y regresión Lasso) y otros que lo han reducido significativamente (como KNN, árboles de decisión y SVM), pero ninguno ha empeorado.

Por último, podríamos extraer atributos relevantes dependiendo del modelo utilizado, ya que algunos (como los árboles de decisión) proporcionan información sobre la importancia de los atributos. Por ejemplo, se puede obtener la importancia de los atributos en uno de estos árboles mediante el atributo feature_importances_.

### Validación cruzada

In [ ]:
from sklearn.model_selection import cross_val_score

# Definición de modelos
models = {
    'KNN': KNeighborsRegressor(),
    'Árboles de regresión': DecisionTreeRegressor(),
    'Regresión lineal': LinearRegression(),
    'Lasso': Lasso(),
    'SVM': SVR()
}

# Realiza la validación cruzada para cada modelo
for model_name, model in models.items():
    scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
    print(f'Modelo: {model_name}')
    print(f'Puntuaciones de validación cruzada: {scores}')
    print(f'Puntuación media de validación cruzada: {scores.mean()}')
    print()

Con respecto a los resultados obtenidos en la validación cruzada podemos ver que KNN tiene una media de 0.823 lo que indica un buen rendimiento. Modelos como árboles de regresión, lasso o regresión lineal nos dan una media de 1 lo que nos puede estar indicando que el modelo esta sobreajustado o está realizandolo muy bien. Mientras que SVM obtiene una puntuación negativa indicando un mal rendimiento.